In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40714 sha256=177b1f198408fb71fbe9fca919b21fc89574d1739802b5dfc35470ec42d3f5f2
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps
Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import print_function
from itertools import combinations
import googlemaps
import pandas as pd
import numpy as np
import os.path
import random
import webbrowser

# Google Maps API key needs to be added here.
GOOGLE_MAPS_API_KEY = "Add you own API"

# Path to the waypoints file.
waypoints_file = "/kaggle/input/my-waypoints-dist-dur/my-waypoints-dist-dur.tsv"

# Base filename for the HTML output. A shorter route's fitness score will be appended.
output_file = 'Output.html'

# Parameters for the genetic algorithm.
thisRunGenerations = 5000
thisRunPopulation_size = 100

# List of all waypoints (50 states + D.C.).
all_waypoints = ["USS Alabama, Battleship Parkway, Mobile, AL",
                 "Grand Canyon National Park, Arizona",
                 "Toltec Mounds, Scott, AR",
                 "San Andreas Fault, San Benito County, CA",
                 "Cable Car Museum, 94108, 1201 Mason St, San Francisco, CA 94108",
                 "Pikes Peak, Colorado",
                 "The Mark Twain House & Museum, Farmington Avenue, Hartford, CT",
                 "New Castle Historic District, Delaware",
                 "White House, Pennsylvania Avenue Northwest, Washington, DC",
                 "Cape Canaveral, FL",
                 "Okefenokee Swamp Park, Okefenokee Swamp Park Road, Waycross, GA",
                 "Craters of the Moon National Monument & Preserve, Arco, ID",
                 "Lincoln Home National Historic Site Visitor Center, 426 South 7th Street, Springfield, IL",
                 "West Baden Springs Hotel, West Baden Avenue, West Baden Springs, IN",
                 "Terrace Hill, Grand Avenue, Des Moines, IA",
                 "C. W. Parker Carousel Museum, South Esplanade Street, Leavenworth, KS",
                 "Mammoth Cave National Park, Mammoth Cave Pkwy, Mammoth Cave, KY",
                 "French Quarter, New Orleans, LA",
                 "Acadia National Park, Maine",
                 "Maryland State House, 100 State Cir, Annapolis, MD 21401",
                 "USS Constitution, Boston, MA",
                 "Olympia Entertainment, Woodward Avenue, Detroit, MI",
                 "Fort Snelling, Tower Avenue, Saint Paul, MN",
                 "Vicksburg National Military Park, Clay Street, Vicksburg, MS",
                 "Gateway Arch, Washington Avenue, St Louis, MO",
                 "Glacier National Park, West Glacier, MT",
                 "Ashfall Fossil Bed, Royal, NE",
                 "Hoover Dam, NV",
                 "Omni Mount Washington Resort, Mount Washington Hotel Road, Bretton Woods, NH",
                 "Congress Hall, Congress Place, Cape May, NJ 08204",
                 "Carlsbad Caverns National Park, Carlsbad, NM",
                 "Statue of Liberty, Liberty Island, NYC, NY",
                 "Wright Brothers National Memorial Visitor Center, Manteo, NC",
                 "Fort Union Trading Post National Historic Site, Williston, North Dakota 1804, ND",
                 "Spring Grove Cemetery, Spring Grove Avenue, Cincinnati, OH",
                 "Chickasaw National Recreation Area, 1008 W 2nd St, Sulphur, OK 73086",
                 "Columbia River Gorge National Scenic Area, Oregon",
                 "Liberty Bell, 6th Street, Philadelphia, PA",
                 "The Breakers, Ochre Point Avenue, Newport, RI",
                 "Fort Sumter National Monument, Sullivan's Island, SC",
                 "Mount Rushmore National Memorial, South Dakota 244, Keystone, SD",
                 "Graceland, Elvis Presley Boulevard, Memphis, TN",
                 "The Alamo, Alamo Plaza, San Antonio, TX",
                 "Bryce Canyon National Park, Hwy 63, Bryce, UT",
                 "Shelburne Farms, Harbor Road, Shelburne, VT",
                 "Mount Vernon, Fairfax County, Virginia",
                 "Hanford Site, Benton County, WA",
                 "Lost World Caverns, Lewisburg, WV",
                 "Taliesin, County Road C, Spring Green, Wisconsin",
                 "Yellowstone National Park, WY 82190"]

def CreateOptimalRouteHtmlFile(optimal_route, distance, display=True):
    """
    Dynamically creates and opens an HTML file displaying the route using Google Maps.
    
    Args:
        optimal_route (tuple): The sequence of waypoints for the route.
        distance (float): The total distance of the route.
        display (bool): If True, opens the HTML file in a web browser.
    """
    optimal_route = list(optimal_route)
    optimal_route += [optimal_route[0]]

    # HTML content for the Google Maps display.
    Page_1 = """
    <!DOCTYPE html>
    <html lang="en">
      <head>
        <meta charset="utf-8">
        <meta name="viewport" content="initial-scale=1.0, user-scalable=no">
        <meta name="description" content="Optimal road trip across the U.S. using a genetic algorithm.">
        <meta name="author" content="Program">
        
        <title>The optimal road trip across the U.S. according to a genetic algorithm</title>
        <style>
          html, body, #map-canvas {
            height: 100%;
            margin: 0px;
            padding: 0px
          }
          #panel {
            position: absolute;
            top: 5px;
            left: 50%;
            margin-left: -180px;
            z-index: 5;
            background-color: #fff;
            padding: 10px;
            border: 1px solid #999;
          }
        </style>
        <script src="https://maps.googleapis.com/maps/api/js?v=3.exp&signed_in=true"></script>
        <script>
            var routes_list = []
            var markerOptions = {icon: "http://maps.gstatic.com/mapfiles/markers2/marker.png"};
            var directionsDisplayOptions = {preserveViewport: true,
                                            markerOptions: markerOptions};
            var directionsService = new google.maps.DirectionsService();
            var map;

            function initialize() {
              var center = new google.maps.LatLng(39, -96);
              var mapOptions = {
                zoom: 5,
                center: center
              };
              map = new google.maps.Map(document.getElementById('map-canvas'), mapOptions);
              for (i=0; i<routes_list.length; i++) {
                routes_list[i].setMap(map); 
              }
            }

            function calcRoute(start, end, routes) {
              
              var directionsDisplay = new google.maps.DirectionsRenderer(directionsDisplayOptions);

              var waypts = [];
              for (var i = 0; i < routes.length; i++) {
                waypts.push({
                  location:routes[i],
                  stopover:true});
                }
              
              var request = {
                  origin: start,
                  destination: end,
                  waypoints: waypts,
                  optimizeWaypoints: false,
                  travelMode: google.maps.TravelMode.DRIVING
              };

              directionsService.route(request, function(response, status) {
                if (status == google.maps.DirectionsStatus.OK) {
                    directionsDisplay.setDirections(response);      
                }
              });

              routes_list.push(directionsDisplay);
            }

            function createRoutes(route) {
                // Google's free map API is limited to 10 waypoints, so need to break into batches.
                route.push(route[0]);
                var subset = 0;
                while (subset < route.length) {
                    var waypointSubset = route.slice(subset, subset + 10);

                    var startPoint = waypointSubset[0];
                    var midPoints = waypointSubset.slice(1, waypointSubset.length - 1);
                    var endPoint = waypointSubset[waypointSubset.length - 1];

                    calcRoute(startPoint, endPoint, midPoints);

                    subset += 9;
                }
            }
    """
    Page_2 = """
            
            createRoutes(optimal_route);

            google.maps.event.addDomListener(window, 'load', initialize);

        </script>
      </head>
      <body>
        <div id="map-canvas"></div>
      </body>
    </html>
    """

    localoutput_file = output_file.replace('.html', '_' + str(distance) + '.html')
    with open(localoutput_file, 'w') as fs:
        fs.write(Page_1)
        fs.write("\t\t\toptimal_route = {0}".format(str(optimal_route)))
        fs.write(Page_2)

    if display:
        webbrowser.open_new_tab(localoutput_file)

def compute_fitness(solution):
    """
    Calculates the total distance of a given road trip (solution).
    The genetic algorithm aims to minimize this value.
    
    Args:
        solution (tuple): A sequence of waypoints.
    
    Returns:
        float: The total distance of the route in meters.
    """
    
    solution_fitness = 0.0
    
    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        
    return solution_fitness

def generate_random_agent():
    """
    Creates a random permutation of waypoints.
    
    Returns:
        tuple: A randomly ordered sequence of all waypoints.
    """
    
    new_random_agent = list(all_waypoints)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)

def mutate_agent(agent_genome, max_mutations=3):
    """
    Applies point mutations to a road trip. A point mutation swaps two waypoints.
    
    Args:
        agent_genome (tuple): The sequence of waypoints to mutate.
        max_mutations (int): The maximum number of swaps to perform.
        
    Returns:
        tuple: The mutated sequence of waypoints.
    """
    
    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)
    
    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
            
    return tuple(agent_genome)

def shuffle_mutation(agent_genome):
    """
    Applies a shuffle mutation. A random sub-section of the route is moved to another location.
    
    Args:
        agent_genome (tuple): The sequence of waypoints to mutate.
        
    Returns:
        tuple: The mutated sequence of waypoints.
    """
    
    agent_genome = list(agent_genome)
    
    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)
    
    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]
    
    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]
    
    return tuple(agent_genome)

def generate_random_population(pop_size):
    """
    Generates an initial population of random road trips.
    
    Args:
        pop_size (int): The desired size of the population.
        
    Returns:
        list: A list of randomly generated waypoint sequences.
    """
    
    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent())
    return random_population
    
def run_genetic_algorithm(generations=5000, population_size=100):
    """
    Executes the genetic algorithm to find the shortest route.
    
    Args:
        generations (int): The number of generations to run the algorithm.
        population_size (int): The number of solutions in each generation.
        
    Returns:
        tuple: The best route found.
    """
    
    current_best_distance = -1
    population_subset_size = int(population_size / 10.)
    generations_10pct = int(generations / 10.)
    
    # Create an initial random population.
    population = generate_random_population(population_size)

    # Main loop for generations.
    for generation in range(generations):
        
        # Compute fitness for the entire population.
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue
            population_fitness[agent_genome] = compute_fitness(agent_genome)

        # Select the top 10% fittest individuals to create the next generation.
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness,
                                                   key=population_fitness.get)[:population_subset_size]):
            # Display progress and save the best route periodically.
            if (generation % generations_10pct == 0 or generation == generations - 1) and rank == 0:
                current_best_genome = agent_genome
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                print(agent_genome)              
                print("")

                if population_fitness[agent_genome] < current_best_distance or current_best_distance < 0:
                    current_best_distance = population_fitness[agent_genome]
                    CreateOptimalRouteHtmlFile(agent_genome, current_best_distance, False)
                    

            # Create offspring from the fittest individuals.
            new_population.append(agent_genome)
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, 3))
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new generation.
        for i in range(len(population))[::-1]:
            del population[i]
        population = new_population
    
    return current_best_genome


if __name__ == '__main__':
    print("Begin finding shortest route")
    file_path = waypoints_file
    
    # Check if a pre-computed waypoints file exists.
    if os.path.exists(file_path):
        print("Waypoints file found. Loading data.")
        waypoint_distances = {}
        waypoint_durations = {}
        all_waypoints = set()

        waypoint_data = pd.read_csv(file_path, sep="\t")

        for i, row in waypoint_data.iterrows():
            waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
            waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
            all_waypoints.update([row.waypoint1, row.waypoint2])
    else:
        # If no file exists, use Google Maps API to compute distances.
        print("Waypoints file not found. Collecting data from Google Maps.")
        waypoint_distances = {}
        waypoint_durations = {}

        gmaps = googlemaps.Client(GOOGLE_MAPS_API_KEY)
        for (waypoint1, waypoint2) in combinations(all_waypoints, 2):
            try:
                route = gmaps.distance_matrix(origins=[waypoint1],
                                              destinations=[waypoint2],
                                              mode="driving",
                                              language="English",
                                              units="metric")
                distance = route["rows"][0]["elements"][0]["distance"]["value"]
                duration = route["rows"][0]["elements"][0]["duration"]["value"]

                waypoint_distances[frozenset([waypoint1, waypoint2])] = distance
                waypoint_durations[frozenset([waypoint1, waypoint2])] = duration
            
            except Exception as e:
                print("Error with finding the route between %s and %s." % (waypoint1, waypoint2))
        
        # Save the computed data to a file for future use.
        print("Saving waypoints data.")
        with open(waypoints_file, "w") as out_file:
            out_file.write("\t".join(["waypoint1",
                                     "waypoint2",
                                     "distance_m",
                                     "duration_s"]))
        
            for (waypoint1, waypoint2) in waypoint_distances.keys():
                out_file.write("\n" +
                               "\t".join([waypoint1,
                                         waypoint2,
                                         str(waypoint_distances[frozenset([waypoint1, waypoint2])]),
                                         str(waypoint_durations[frozenset([waypoint1, waypoint2])])]))

    # Run the genetic algorithm to find the optimal route.
    print("Searching for optimal route.")
    optimal_route = run_genetic_algorithm(generations=thisRunGenerations, population_size=thisRunPopulation_size)

    # Create and display the HTML file for the final best route.
    CreateOptimalRouteHtmlFile(optimal_route, 1, True)

Begin finding shortest route
Waypoints file found. Loading data.
Searching for optimal route.
Generation 0 best: 89620752 | Unique genomes: 100
('Graceland, Elvis Presley Boulevard, Memphis, TN', 'New Castle Historic District, Delaware', 'Spring Grove Cemetery, Spring Grove Avenue, Cincinnati, OH', 'Okefenokee Swamp Park, Okefenokee Swamp Park Road, Waycross, GA', 'Lost World Caverns, Lewisburg, WV', 'Maryland State House, 100 State Cir, Annapolis, MD 21401', 'Terrace Hill, Grand Avenue, Des Moines, IA', 'Fort Snelling, Tower Avenue, Saint Paul, MN', "Fort Sumter National Monument, Sullivan's Island, SC", 'French Quarter, New Orleans, LA', 'San Andreas Fault, San Benito County, CA', 'Mount Vernon, Fairfax County, Virginia', 'USS Constitution, Boston, MA', 'C. W. Parker Carousel Museum, South Esplanade Street, Leavenworth, KS', 'Cape Canaveral, FL', 'Congress Hall, Congress Place, Cape May, NJ 08204', 'Toltec Mounds, Scott, AR', 'USS Alabama, Battleship Parkway, Mobile, AL', 'Yellowston